In [33]:
import pandas
import numpy as np
import os
import shutil
from pathlib import Path
import json
import math
from PIL import Image, ImageDraw
import glob

import torch
import segmentation_models_pytorch as smp
import cv2
import torch
from torch.utils.data import Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split


In [13]:
device = torch.device("mps" if torch.backends.mps.is_available() else
                      "cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")    

Using device: mps


In [ ]:
SEED = 42

In [14]:
data_dir = Path().cwd().parent / "dataset"
if not data_dir.exists():
    raise FileNotFoundError(f"Data directory {data_dir} does not exist.")

images_dir = data_dir / "images"
if not images_dir.exists():
    raise FileNotFoundError(f"Images directory {images_dir} does not exist.")
labels_dir = data_dir / "labels"
if not labels_dir.exists():
    raise FileNotFoundError(f"Labels directory {labels_dir} does not exist.")
masks_dir = data_dir / "masks"
if not masks_dir.exists():
    raise FileNotFoundError(f"Masks directory {masks_dir} does not exist.")

# images_dir.mkdir(exist_ok=True)
# labels_dir.mkdir(exist_ok=True)
# masks_dir.mkdir(exist_ok=True)

print(f"Data directory: {data_dir}")
print(f"Images directory: {images_dir}")
print(f"Labels directory: {labels_dir}")

Data directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset
Images directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/images
Labels directory: /Users/ratchanonkhongsawi/Desktop/CMKL/3rd/Computer Vision/project/Computer-vision-2025/dataset/labels


In [15]:
all_labels = set()

for jp in labels_dir.glob("*.json"):
    with open(jp, "r") as f:
        anno = json.load(f)
    for shape in anno.get("shapes", []):
        lbl = shape.get("label", "")
        all_labels.add(lbl)

print("Found labels:", all_labels)

Found labels: {'Nerve', 'nerve'}


In [16]:
CLASS_MAP = {"background": 0, "nerve": 1}

In [35]:
jpg_files = glob.glob(str(images_dir / "*.jpg")) + glob.glob(str(images_dir / "*.JPG"))

for f in jpg_files:
    path = Path(f)
    try:
        path.unlink()  # remove file
        print(f"🗑️ Removed {path.name}")
    except Exception as e:
        print(f"⚠️ Could not remove {path.name}: {e}")

🗑️ Removed 189.jpg
🗑️ Removed 162.jpg
🗑️ Removed 176.jpg
🗑️ Removed 228.jpg
🗑️ Removed 214.jpg
🗑️ Removed 200.jpg
🗑️ Removed 201.jpg
🗑️ Removed 215.jpg
🗑️ Removed 229.jpg
🗑️ Removed 177.jpg
🗑️ Removed 163.jpg
🗑️ Removed 188.jpg
🗑️ Removed 149.jpg
🗑️ Removed 175.jpg
🗑️ Removed 161.jpg
🗑️ Removed 203.jpg
🗑️ Removed 217.jpg
🗑️ Removed 216.jpg
🗑️ Removed 202.jpg
🗑️ Removed 160.jpg
🗑️ Removed 174.jpg
🗑️ Removed 148.jpg
🗑️ Removed 170.jpg
🗑️ Removed 164.jpg
🗑️ Removed 158.jpg
🗑️ Removed 206.jpg
🗑️ Removed 212.jpg
🗑️ Removed 213.jpg
🗑️ Removed 207.jpg
🗑️ Removed 159.jpg
🗑️ Removed 165.jpg
🗑️ Removed 171.jpg
🗑️ Removed 198.jpg
🗑️ Removed 167.jpg
🗑️ Removed 173.jpg
🗑️ Removed 211.jpg
🗑️ Removed 205.jpg
🗑️ Removed 239.jpg
🗑️ Removed 238.jpg
🗑️ Removed 204.jpg
🗑️ Removed 210.jpg
🗑️ Removed 172.jpg
🗑️ Removed 166.jpg
🗑️ Removed 199.jpg
🗑️ Removed 129.jpg
🗑️ Removed 115.jpg
🗑️ Removed 288.jpg
🗑️ Removed 277.jpg
🗑️ Removed 263.jpg
🗑️ Removed 262.jpg
🗑️ Removed 276.jpg
🗑️ Removed 289.jpg
🗑️ Removed 1

In [ ]:
image_shapes = []
mask_shapes = []

for image in (images_dir.glob("*.png")):
    img = np.array(Image.open(image))
    image_shapes.append(img.shape[:2])  # (h, w)

for mask in masks_dir.glob("*.png"):
    m = np.array(Image.open(mask))
    mask_shapes.append(m.shape[:2])  # (h, w)

# check consistency
unique_image_shapes = set(image_shapes)
unique_mask_shapes = set(mask_shapes)

if len(unique_image_shapes) > 1:
    print("⚠️ Not all images have the same size:", unique_image_shapes)

if len(unique_mask_shapes) > 1:
    print("⚠️ Not all masks have the same size:", unique_mask_shapes)

if unique_image_shapes != unique_mask_shapes:
    print("⚠️ Image and mask sizes differ.")
else:
    print("✅ All images and masks have the same size:", unique_image_shapes.pop())

TypeError: unsupported operand type(s) for +: 'generator' and 'generator'

In [ ]:
train_imgs, val_imgs, train_masks, val_masks = train_test_split(
    images_dir, masks_dir, test_size=0.2, random_state=SEED
)

In [ ]:
pad_to_32 = A.PadIfNeeded(
    min_height=None, min_width=None,
    pad_height_divisor=32, pad_width_divisor=32,
    border_mode=cv2.BORDER_CONSTANT,  # pad with black for image, 0 for mask
    value=0, mask_value=0
)

train_transform = A.Compose([
    pad_to_32,
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    pad_to_32,
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])